In [ ]:
# import sys
# !{sys.executable} -m pip install matplotlib --user
# !{sys.executable} -m pip install hyperopt --user

In [1]:
from time import time
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from copy import deepcopy
from sklearn.neural_network import MLPClassifier
import numpy as np

# Data science imports
import pandas as pd
from enum import Enum
import lightgbm
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn_pandas import DataFrameMapper

# Visualisation imports
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# File IO: Read the data file
file = open('train_data.txt', mode = 'r')
lines = file.readlines()
file.close()

In [3]:
# Check for completeness
len(lines)

78945

In [4]:
# Pre-processing: Clean unwanted characters 
data = []
i = 0
while i < len(lines)-1:
    row = {}
    row['id'] = lines[i+1].split(" ")[1].strip()
    row['durationOfStay'] = lines[i+2].split(" ")[1].strip()
    row['gender'] = lines[i+3].split(" ")[1].strip()
    row['Age'] = lines[i+4].split(" ")[1].strip()
    row['kids'] = lines[i+5].split(" ")[1].strip()
    row['destinationCode'] = lines[i+6].split(" ")[1].strip()
    row['AcomType'] = lines[i+7].split(" ")[1].strip()
    data.append(row)
    i = i + 8

In [5]:
# Structure the data
def structure_data_to_df(data, test_data=False):
    if test_data:
        df = pd.DataFrame(data, columns=['id','durationOfStay','gender','Age','kids','destinationCode'])
        df['id'] = df['id'].astype(str)
    else:
        df = pd.DataFrame(data, columns=['durationOfStay','gender','Age','kids','destinationCode','AcomType'])
        df['AcomType'] = df['AcomType'].astype(str)
        
    df['gender'] = df['gender'].astype(str)
    df['kids'] = df['kids'].astype(str)
    df['destinationCode'] = df['destinationCode'].astype(str)
    df = df.replace('<NA>', np.nan)
    df = df.replace('NA', np.nan)
    df = df.replace('nan', np.nan)
    df[['durationOfStay', 'Age']] = df[['durationOfStay', 'Age']].apply(pd.to_numeric)
    
    return df
    
df = structure_data_to_df(data)

In [6]:
# utility variables
categorical_cols = ['gender']
missing_categorical_cols = ['kids', 'destinationCode']
numerical_cols = ['durationOfStay']
missing_numerical_cols = ['Age']

# transform utility function
def build_imputer_and_transformer(missing_categorical_cols, categorical_cols, 
                                  missing_numerical_cols, numerical_cols, constant=True):
    transformations = []
    
    for col in missing_categorical_cols:
        if constant:
            transformations.append(([col], [SimpleImputer(strategy='constant', fill_value='N/A'), LabelBinarizer()]))
        else:
            transformations.append(([col], [SimpleImputer(strategy='most_frequent'), LabelBinarizer()]))
        
    for col in categorical_cols:
        transformations.append((col, LabelBinarizer()))
        
    transformations.append((missing_numerical_cols, [SimpleImputer(strategy='mean'), StandardScaler()]))
    transformations.append((numerical_cols, StandardScaler()))
   
    transformer = DataFrameMapper(transformations)
    return transformer

In [7]:
y = df['AcomType'].copy()
X = df.drop(['AcomType'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)
    
transformer = build_imputer_and_transformer(missing_categorical_cols, categorical_cols,
                                             missing_numerical_cols, numerical_cols, constant=True)
    
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)
y_label_encoder = LabelEncoder()
y_train = y_label_encoder.fit_transform(y_train)

lightdbm_dataset = lightgbm.Dataset(X_train, label=y_train)

In [8]:
boosting_type_list = ['dart', 'gbdt', 'rf']

In [9]:
def optimize(evals, trials, optimizer=tpe.suggest):      
    space = {
        'class_weight': hp.choice('class_weight', [None, 'balanced']),
        'boosting_type': hp.choice('boosting_type', boosting_type_list),
        'num_threads': 20,
        'n_estimators': hp.choice('n_estimators', np.arange(40, 1000, dtype=int)),
        'num_leaves': hp.choice('num_leaves', np.arange(30, 150, dtype=int)),
#          'num_leaves': hp.choice('num_leaves', num_leaves_list),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
#         'subsample_for_bin': hp.choice('subsample_for_bin', np.arange(20000, 300000, dtype=int)),
        'min_child_samples': hp.choice('min_child_samples', np.arange(20, 500, dtype=int)),
        'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
        'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
        'bagging_freq': hp.choice('bagging_freq', np.arange(1, 10, dtype=int)),
        'bagging_fraction': 0.8
    }
    best = fmin(score, space, algo=optimizer, max_evals=evals, trials=trials)
    pbar.close()
    return best

In [10]:
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []

def score(params):
    
    print(params)
    results = lightgbm.cv(params, lightdbm_dataset, nfold=5, metrics='binary_error', early_stopping_rounds=10)
    scores_mean = results['binary_error-mean'][-1]
    print('Parameters with this training score {} :'.format(scores_mean))
    pbar.update()
#     return {'loss': test_loss, 'status': STATUS_OK}
    return {'loss': scores_mean, 'status': STATUS_OK}
     

In [11]:
import time
from tqdm import tqdm

trials = Trials()
cores = 60
start = time.time()
evaluations = 1000
pbar = tqdm(total=evaluations, desc="Hyperopt")
best_param = optimize(evals=evaluations,
                      optimizer=tpe.suggest,
                      trials=trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))

Hyperopt:   0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]
{'bagging_fraction': 0.8, 'bagging_freq': 6, 'boosting_type': 'gbdt', 'class_weight': 'balanced', 'colsample_bytree': 0.7647768171583315, 'learning_rate': 0.0876642585782813, 'min_child_samples': 484, 'n_estimators': 569, 'num_leaves': 145, 'num_threads': 20, 'reg_alpha': 0.4424149577036891, 'reg_lambda': 0.8627464793549998}

  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]
Parameters with this training score 0.21167794321291256 :

  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]

Hyperopt:   0%|          | 1/1000 [00:01<21:47,  1.31s/it]


  0%|          | 1/1000 [00:00<11:38,  1.43it/s, best loss: 0.21167794321291256]
{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'dart', 'class_weight': 'balanced', 'colsample_bytree': 0.6403531858009428, 'learning_rate': 0.04751012266703769, 'min_child_samples': 171, 'n_estimators': 138, 'num_leaves': 34, 'num_threads': 20, 'reg_alpha': 0.5813332701000861, 'reg_lambda': 0.9975178919629405}

  0%|          | 1/1000 [00:00<11:38,  1.43it/s, best loss: 0.21167794321291256]
Parameters with this training score 0.4333671153702715 :

  0%|          | 1/1000 [00:01<11:38,  1.43it/s, best loss: 0.21167794321291256]

Hyperopt:   0%|          | 2/1000 [00:02<20:40,  1.24s/it]

...


100%|█████████▉| 998/1000 [26:24<00:02,  1.06s/it, best loss: 0.20635972284637144]
{'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9263100111897953, 'learning_rate': 0.1389907655391424, 'min_child_samples': 118, 'n_estimators': 161, 'num_leaves': 127, 'num_threads': 20, 'reg_alpha': 0.8137910256397201, 'reg_lambda': 0.13591900006787305}

100%|█████████▉| 998/1000 [26:24<00:02,  1.06s/it, best loss: 0.20635972284637144]
Parameters with this training score 0.2128208733921273 :

100%|█████████▉| 998/1000 [26:25<00:02,  1.06s/it, best loss: 0.20635972284637144]

Hyperopt: 100%|█████████▉| 999/1000 [26:26<00:01,  1.02s/it]


100%|█████████▉| 999/1000 [26:25<00:01,  1.02s/it, best loss: 0.20635972284637144]
{'bagging_fraction': 0.8, 'bagging_freq': 6, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9777878852001057, 'learning_rate': 0.15853526586162098, 'min_child_samples': 481, 'n_estimators': 861, 'num_leaves': 122, 'num_threads': 20, 'reg_alpha': 0.6826397490179585, 'reg_lambda': 0.055476200513909155}

100%|█████████▉| 999/1000 [26:25<00:01,  1.02s/it, best loss: 0.20635972284637144]
Parameters with this training score 0.24638931084471918 :

100%|█████████▉| 999/1000 [26:26<00:01,  1.02s/it, best loss: 0.20635972284637144]

Hyperopt: 100%|██████████| 1000/1000 [26:26<00:00,  1.19it/s]


100%|██████████| 1000/1000 [26:26<00:00,  1.19it/s, best loss: 0.20635972284637144]------------------------------------
The best hyperparameters are:  

{'bagging_freq': 0, 'boosting_type': 1, 'class_weight': 0, 'colsample_by_tree': 0.9729252251074288, 'learning_rate': 0.13994677714773102, 'min_child_samples': 395, 'n_estimators': 729, 'num_leaves': 72, 'reg_alpha': 0.7523042693713519, 'reg_lambda': 0.8452353677057264}
Time elapsed to optimize 1000 executions: 1586.7095293998718


In [1]:
# class_weights = [None, 'balanced']
# best_param['boosting_type'] = boosting_type_list[best_param['boosting_type']]
# best_param['class_weight'] = class_weights[best_param['class_weight']]
best_param = {'bagging_fraction': 0.8, 'bagging_freq': 1, 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9729252251074288, 'learning_rate': 0.13994677714773102, 'min_child_samples': 415, 'n_estimators': 769, 'num_leaves': 102, 'num_threads': 20, 'reg_alpha': 0.7523042693713519, 'reg_lambda': 0.8452353677057264}

print('\n Best score:')
score(best_param)


 Best score:


NameError: name 'score' is not defined

In [14]:
# get the best params and train the model
# from lightgbm import LGBMClassifier
LBGM_opt = lightgbm.LGBMClassifier(class_weight=best_param['class_weight'], boosting_type=best_param['boosting_type'], 
                          num_leaves=best_param['num_leaves'],
                                  learning_rate=best_param['learning_rate'], 
#                           subsample_for_bin=best_param['subsample_for_bin'], 
                          min_child_samples=best_param['min_child_samples'],
                                  reg_alpha=best_param['reg_alpha'], 
                                   n_estimators=best_param['n_estimators'],
                          reg_lambda=best_param['reg_lambda'], colsample_by_tree=best_param['colsample_by_tree'],
                                  random_state=42, n_jobs=48)
# X = transformer.fit_transform(X)

fitted_model = LBGM_opt.fit(X=X_train,y=y_train)

In [15]:
y_score = fitted_model.predict_proba(X_test)
y_score

array([[0.34993219, 0.65006781],
       [0.71365908, 0.28634092],
       [0.15974084, 0.84025916],
       ...,
       [0.10249884, 0.89750116],
       [0.58816836, 0.41183164],
       [0.49905821, 0.50094179]])

In [ ]:
test_acc = accuracy_score(y_label_encoder.transform(y_test), fitted_model.predict(X_test))
test_acc

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline 
x_list =range(len(train_acc_list))

plt.plot(x_list, train_acc_list, label='Training accuracy')
plt.plot(x_list, test_acc_list, label='Testing accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_list, train_loss_list, label='Training loss')
plt.plot(x_list, test_loss_list, label='Testing loss')
plt.legend()
plt.show()